In [1]:
import spark
from pyspark import SparkContext
from pyspark.sql import functions as F
from pyspark.sql.functions import col, split
from pyspark.sql import SQLContext
import json

In [2]:
sc = SparkContext("local","textfile")
sqlContext = SQLContext(sc)

In [3]:
filepath = "/home/saeed/Documents/spark_project/src/Businesses_London_0-1000_offset.json"
f = open(filepath,"r")
json_data = f.read()
response_list = json.loads(json_data)


In [4]:
# Remove the headers leaving only data we are interested in
# The headers in this case are the containers that house our data
# Look within the businesses header and extract the data within
# Data architecture
#    json file
#        [businesses - id, alias, name .... (very interested)
#         region - Not interested
#         total - Not interested
#        ]
#         ...
#        [businesses - id, alias, name .... (very interested)
#         region - Not interested
#         total - Not interested
#        ]
businesses_data_list = []
for response_data in response_list:
    for business_data in response_data["businesses"]:
        businesses_data_list.append(business_data)

businesses_data_list[:5]

[{'id': 'oK_SLmmAVQg3meguh7LrIA',
  'alias': 'dishoom-london',
  'name': 'Dishoom',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/4HC2FmxHZYzxqlqiRaeKCQ/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/dishoom-london?adjust_creative=ZwkqKwSASAVHCFsz3-oZmQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=ZwkqKwSASAVHCFsz3-oZmQ',
  'review_count': 1843,
  'categories': [{'alias': 'indpak', 'title': 'Indian'}],
  'rating': 4.5,
  'coordinates': {'latitude': 51.512447030091, 'longitude': -0.1269579217873},
  'transactions': [],
  'price': '££',
  'location': {'address1': "12 Upper Saint Martin's Lane",
   'address2': '',
   'address3': '',
   'city': 'London',
   'zip_code': 'WC2H 9FB',
   'country': 'GB',
   'state': 'XGL',
   'display_address': ["12 Upper Saint Martin's Lane",
    'London WC2H 9FB',
    'United Kingdom']},
  'phone': '+442074209320',
  'display_phone': '+44 20 7420 9320',
  'distance': 1112.4775110385249},
 {'id': 'YRA2VnhFL

In [5]:
len(businesses_data_list)

999

In [6]:
businesses_data_json = json.dumps(businesses_data_list)
#businesses_data_json

In [7]:
#data = json.load(open(filepath,"r"))
json_rdd = sc.parallelize([businesses_data_json])
#json_rdd.take(1)

In [8]:
df = sqlContext.read.json(json_rdd)

In [9]:
df.show()

+--------------------+--------------------+--------------------+----------------+------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------+-----+------+------------+------------+--------------------+
|               alias|          categories|         coordinates|   display_phone|          distance|                  id|           image_url|is_closed|            location|                name|         phone|price|rating|review_count|transactions|                 url|
+--------------------+--------------------+--------------------+----------------+------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------+-----+------+------------+------------+--------------------+
|      dishoom-london|  [[indpak, Indian]]|[51.512447030091,...|+44 20 7420 9320|1112.4775110385249|oK_SLmmAVQg3meguh...|https://s3-media2...|    false|[12 Upper Saint M...|             Dish

In [10]:
df.printSchema()

root
 |-- alias: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- alias: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |-- display_phone: string (nullable = true)
 |-- distance: double (nullable = true)
 |-- id: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- is_closed: boolean (nullable = true)
 |-- location: struct (nullable = true)
 |    |-- address1: string (nullable = true)
 |    |-- address2: string (nullable = true)
 |    |-- address3: string (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- display_address: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- state: string (nullable = true)
 |    |-- zip_code: string (nulla

In [11]:
#df.select("alias").show()
#df.select("categories").show()
df.select("price").show()

+-----+
|price|
+-----+
|   ££|
| null|
| null|
| null|
| null|
|   ££|
| null|
|   ££|
| null|
|   ££|
| null|
|   ££|
| null|
| null|
|  £££|
|   ££|
|  £££|
|    £|
| null|
|   ££|
+-----+
only showing top 20 rows

